# Proyecto de campaña de marketing para Banco


Algunos puntos a considerar:
- Objetivo: se quiere predecir si un cliente del Banco va a realizar un depósito a plazo o no -> Predicción
- Los datos están etiquetados, es decir, tenemos una muestra del comportamiento observado de algunos usuarios -> Aprendizaje Supervisado
- La variable a explicar (y : ¿Va a depositar a plazo?) es categórica (si/no) -> Problema de clasificación

### Paso 1 : Importación de datos desde página web

In [2]:
import pandas as pd
import numpy as np

In [3]:
df=pd.read_csv('https://raw.githubusercontent.com/4GeeksAcademy/logistic-regression-project-tutorial/main/bank-marketing-campaign-data.csv',sep=';')

### Paso 2 : Análisis exploratorio de datos


Lo primero es ver cómo vienen los datos, para ello se imprime una muestra aleatoria de 10 observaciones y se analiza la información principal y descriptiva de la base.

In [4]:
df.sample(10)

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
17394,57,technician,married,professional.course,unknown,yes,no,cellular,jul,mon,...,2,999,0,nonexistent,1.4,93.918,-42.7,4.962,5228.1,no
27579,47,admin.,married,high.school,no,yes,no,cellular,nov,fri,...,3,999,0,nonexistent,-0.1,93.200,-42.0,4.021,5195.8,no
11087,33,admin.,single,professional.course,no,no,no,telephone,jun,thu,...,1,999,0,nonexistent,1.4,94.465,-41.8,4.961,5228.1,no
32344,33,blue-collar,single,basic.9y,no,yes,no,cellular,may,fri,...,1,999,1,failure,-1.8,92.893,-46.2,1.313,5099.1,no
21120,33,unemployed,divorced,professional.course,no,no,no,cellular,aug,mon,...,4,999,0,nonexistent,1.4,93.444,-36.1,4.963,5228.1,no
28642,33,blue-collar,married,basic.9y,no,yes,no,cellular,apr,thu,...,1,999,1,failure,-1.8,93.075,-47.1,1.410,5099.1,no
32786,34,unemployed,married,basic.9y,no,no,no,cellular,may,mon,...,2,999,0,nonexistent,-1.8,92.893,-46.2,1.299,5099.1,no
3411,36,management,married,basic.9y,no,yes,no,telephone,may,thu,...,6,999,0,nonexistent,1.1,93.994,-36.4,4.860,5191.0,no
26649,29,technician,single,professional.course,unknown,yes,yes,cellular,nov,thu,...,1,999,1,failure,-0.1,93.200,-42.0,4.076,5195.8,no
26833,39,entrepreneur,married,professional.course,no,unknown,unknown,cellular,nov,thu,...,3,999,0,nonexistent,-0.1,93.200,-42.0,4.076,5195.8,no


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             41188 non-null  int64  
 1   job             41188 non-null  object 
 2   marital         41188 non-null  object 
 3   education       41188 non-null  object 
 4   default         41188 non-null  object 
 5   housing         41188 non-null  object 
 6   loan            41188 non-null  object 
 7   contact         41188 non-null  object 
 8   month           41188 non-null  object 
 9   day_of_week     41188 non-null  object 
 10  duration        41188 non-null  int64  
 11  campaign        41188 non-null  int64  
 12  pdays           41188 non-null  int64  
 13  previous        41188 non-null  int64  
 14  poutcome        41188 non-null  object 
 15  emp.var.rate    41188 non-null  float64
 16  cons.price.idx  41188 non-null  float64
 17  cons.conf.idx   41188 non-null 

Se observa que el dataset cuenta con 41.188 observaciones y 21 variables. A su vez, el tipo de dato de las variables está bien definido y no existen valores nulos en el dataset.

In [6]:
df.describe(include='object')

,job,marital,education,default,housing,loan,contact,month,day_of_week,poutcome,y
count,41188,41188,41188,41188,41188,41188,41188,41188,41188,41188,41188
unique,12,4,8,3,3,3,2,10,5,3,2
top,admin.,married,university.degree,no,yes,no,cellular,may,thu,nonexistent,no
freq,10422,24928,12168,32588,21576,33950,26144,13769,8623,35563,36548


Del resumen anterior se observa la cantidad de clases que tiene cada variable, la clase que predomina y su frecuencia. Así se puede decir que los clientes que trabajan como administradores, están casados, tienen un grado universitario, no son impagos (default), tienen préstamos hipotecarios pero no tienen personales y fueron contactados un miércoles de mayo, tienen mayor probabilidad de no tener un depósito a plazo.

Exploro las categorias posibles de cada variable cualitativa para ver cuáles tienen valores desconocidos.

In [43]:
uk=[]
for i in df:
    [uk.append(i) for n in df[i].unique() if n=='unknown']

print(uk)

['job', 'marital', 'education', 'default', 'housing', 'loan']


In [44]:
for i in uk:
    df[i].replace(to_replace='unknown', value= df[i].mode()) 

ValueError: Series.replace cannot use dict-value and non-None to_replace

In [7]:
round(df.describe(include=['int','float64']),2)

,age,duration,campaign,pdays,previous,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed
count,41188.00,41188.00,41188.00,41188.00,41188.00,41188.00,41188.00,41188.00,41188.00,41188.00
mean,40.02,258.29,2.57,962.48,0.17,0.08,93.58,-40.50,3.62,5167.04
std,10.42,259.28,2.77,186.91,0.49,1.57,0.58,4.63,1.73,72.25
min,17.00,0.00,1.00,0.00,0.00,-3.40,92.20,-50.80,0.63,4963.60
25%,32.00,102.00,1.00,999.00,0.00,-1.80,93.08,-42.70,1.34,5099.10
50%,38.00,180.00,2.00,999.00,0.00,1.10,93.75,-41.80,4.86,5191.00
75%,47.00,319.00,3.00,999.00,0.00,1.40,93.99,-36.40,4.96,5228.10
max,98.00,4918.00,56.00,999.00,7.00,1.40,94.77,-26.90,5.04,5228.10


Se observa que la edad promedio de los clientes es de 40 años y que hay clientes que en la última campaña fueron contactados como máximo 56 veces.

A continuación se observa si existen lineas repetidas en el dataset.

In [8]:
print('Existen ', df.duplicated().sum(),'registros repetidos, se eliminan de la base de datos para que no interfieran en la predicción. Finalmente el dataset contiene ', len(df.drop_duplicates()),' registros únicos.')

df=df.drop_duplicates()

Existen  12 registros repetidos, se eliminan de la base de datos para que no interfieran en la predicción. Finalmente el dataset contiene  41176  registros únicos.


Step 4

In [9]:
X=df.drop(columns=['y'])

In [10]:
y=df['y']

In [11]:
X.select_dtypes(include=['int','float64']).columns

Index(['age', 'duration', 'campaign', 'pdays', 'previous', 'emp.var.rate',
       'cons.price.idx', 'cons.conf.idx', 'euribor3m', 'nr.employed'],
      dtype='object')

In [12]:
#scikit learn logistic regression